In [32]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
json_prepared_filtered = dataiku.Dataset("Json_prepared_filtered")
df = json_prepared_filtered.get_dataframe()

In [33]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [34]:
del df['Timestamp']
df_cpy  = df.set_index('date')

In [35]:
val = df_cpy
scalers={}
for i in df_cpy.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(val[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    val[i]=s_s

In [36]:
val_to_pred = val.values
val_to_pred = val_to_pred.reshape((1, val_to_pred.shape[0],val_to_pred.shape[1]))

In [37]:
handle = dataiku.Folder("models")
json_config = handle.read_json("model_json")
model = keras.models.model_from_json(json_config)

/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [38]:
val_pred = model.predict(val_to_pred)

In [39]:
val = val_pred.squeeze()
val.shape

(5, 30)

In [43]:
pd.DataFrame(val)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-0.167541,0.030422,0.301656,-0.033036,0.004076,0.099365,-0.022584,0.010397,0.160482,0.164269,-0.198799,0.072712,-0.048479,0.174819,-0.021397,-0.079354,-0.010619,-0.222815,0.124031,-0.241170,0.054417,-0.106700,0.108048,0.146577,0.191625,-0.216005,-0.066658,-0.078948,-0.048901,0.040698
1,-0.068319,-0.027517,0.216861,-0.023262,-0.024179,0.070408,-0.008399,-0.000389,0.135149,0.148135,-0.112425,0.022752,-0.021547,0.116353,-0.023335,-0.108245,-0.005611,-0.150221,0.043002,-0.161296,-0.028551,-0.051502,0.085733,0.148392,0.120521,-0.181846,-0.048511,-0.080069,0.006573,-0.003307
2,-0.005438,-0.072222,0.153603,-0.031606,-0.038643,0.054338,0.006429,-0.008563,0.113533,0.132622,-0.055113,-0.019492,-0.012355,0.072268,-0.024759,-0.141195,-0.017231,-0.094167,-0.029770,-0.106632,-0.083285,-0.015446,0.082650,0.146469,0.079261,-0.151268,-0.034119,-0.079868,0.038096,-0.030499
3,0.032302,-0.105706,0.106762,-0.047324,-0.047352,0.049734,0.018531,-0.013406,0.095346,0.120240,-0.019261,-0.051025,-0.011971,0.038554,-0.025485,-0.173726,-0.034776,-0.054923,-0.091352,-0.071669,-0.118178,0.005528,0.090752,0.144697,0.056229,-0.124631,-0.023818,-0.078577,0.056451,-0.046659
4,0.053543,-0.130402,0.071862,-0.064997,-0.053978,0.052225,0.028025,-0.015612,0.079748,0.110985,0.002165,-0.072968,-0.015516,0.012539,-0.025518,-0.202781,-0.052847,-0.029431,-0.141691,-0.050691,-0.139545,0.016782,0.104339,0.143840,0.043736,-0.101971,-0.016788,-0.076559,0.067037,-0.055633


In [40]:
lastdate = df["date"].max()
print(lastdate)
for i in range(len(val)):
    lastdate = lastdate + timedelta(minutes=15)
    row = dict()
    
    row["date"] = lastdate
    
    df.append({})

2021-08-05 11:45:00+00:00


TypeError: append() missing 1 required positional argument: 'other'

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

last_prediction_df = json_prepared_filtered_df # For this sample code, simply copy input to output


# Write recipe outputs
last_prediction = dataiku.Dataset("last_prediction")
last_prediction.write_with_schema(last_prediction_df)